<a href="https://colab.research.google.com/github/DebicharanTripathy/DebicharanTripathy/blob/main/m22ai545_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Author: Debicharan Tripathy
# Roll no: m22ai545

# Task 2: as per GPT Download the data from http://ai.stanford.edu/~acoates/stl10/stl10_binary.tar.gz
# Execute it by updating the path.

import os
import numpy as np
import tensorflow as tf;
import tensorflow as tf from sklearn.metrices
import confusion_matrix, roc_auc_score from sklearn.model_selection
import train_test_split from tensorflow.keras
import layers, models, optimizers from tensorflow.keras.preprocessing.image
import ImageDataGenerator from tensorflow.keras.applications
import VGG16

# Task 2.1: Preprocess of the STL-10 dataset
data_dir = 'stl10_data/'
train_images = np.load(os.path.join(data_dir, 'train_images.npy'))
train_labels = np.load(os.path.join(data_dir, 'train_labels.npy'))
test_images = np.load(os.path.join(data_dir, 'test_images.npy'))
test_labels = np.load(os.path.join(data_dir, 'test_labels.npy'))

# Preprocess of the images (e.g., resizing, normalization)

# Divide the dataset into sets for training and validation.
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Task 2.2: Use the encoder of a trained autoencoder as a feature extractor.
autoencoder = models.load_model('pretrained_autoencoder.h5')
encoder = autoencoder.layers[1]  # assuming that the second layer is the encoder

# Extract features from the images
train_features = encoder.predict(train_images)
val_features = encoder.predict(val_images)
test_features = encoder.predict(test_images)

# Task 2.3: Build MLP classifiers
hidden_layers_3 = [256, 128, 64]
hidden_layers_5 = [512, 256, 128, 64, 32]

def build_mlp_classifier(hidden_layers):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=train_features.shape[1:]))
    for units in hidden_layers:
        model.add(layers.Dense(units, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    return model

# Create an MLP classifier using three hidden layers.
classifier_3 = build_mlp_classifier(hidden_layers_3)
classifier_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier_3.fit(train_features, train_labels, validation_data=(val_features, val_labels), epochs=10)

# Create an MLP classifier using five hidden layers.
classifier_5 = build_mlp_classifier(hidden_layers_5)
classifier_5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier_5.fit(train_features, train_labels, validation_data=(val_features, val_labels), epochs=10)

# Task 2.4: Fine-tune classifiers with the different training sample percentages
train_samples = [0.01, 0.1, 0.2, 0.4, 0.6]
classifiers = [classifier_3, classifier_5]
results = {}

for classifier in classifiers:
    classifier_results = []
    for sample_percent in train_samples:
        # Take a sample at random from the training data.
        sample_indices = np.random.choice(len(train_features), int(sample_percent * len(train_features)), replace=False)
        sampled_features = train_features[sample_indices]
        sampled_labels = train_labels[sample_indices]

        # Classifier refinement using sampling data
        classifier.fit(sampled_features, sampled_labels, epochs=5, verbose=0)

        # Evaluate the classifier on test set
        test_loss, test_acc = classifier.evaluate(test_features, test_labels, verbose=0)

        classifier_results.append(test_acc)

    results[classifier.name] = classifier_results

# Task 2.5: Evaluate the performance using confusion matrix and AUC-ROC curve
def evaluate_performance(classifier, features, labels):
    predictions = classifier.predict(features)
    y_pred = np.argmax(predictions, axis=1)
    cm = confusion_matrix(labels, y_pred)
    auc_roc = roc_auc_score(labels, predictions, multi_class='ovr')
    return cm, auc_roc

# Evaluate the performance for Task 2.3.a and 2.3.b
cm_3, auc_roc_3 = evaluate_performance(classifier_3, test_features, test_labels)
cm_5, auc_roc_5 = evaluate_performance(classifier_5, test_features, test_labels)

# Evaluate the performance for Task 2.4.a-e
cm_results = {}
auc_roc_results = {}

for classifier_name, classifier_results in results.items():
    cm, auc_roc = evaluate_performance(classifiers[classifier_name], test_features, test_labels)
    cm_results[classifier_name] = cm
    auc_roc_results[classifier_name] = auc_roc

# Task 2.6: Implement a different architecture for improvement (e.g., CNN)
def build_cnn_classifier():
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=train_images.shape[1:])
    for layer in base_model.layers:
        layer.trainable = False
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    return model

cnn_classifier = build_cnn_classifier()
cnn_classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_classifier.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=10)

# Analyse the performance of the CNN classifier.
cm_cnn, auc_roc_cnn = evaluate_performance(cnn_classifier, test_images, test_labels)

SyntaxError: ignored